In [59]:
%pip -q install google-genai


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [60]:
# Configura a API Key do Google Gemini

import os
from dotenv import load_dotenv

load_dotenv()

os.environ["GOOGLE_API_KEY"] = os.getenv('GOOGLE_API_KEY')

In [61]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [62]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [63]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [64]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [65]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [66]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_explorador(topico, data_de_hoje):

    explorador = Agent(
        name="agente_explorador",
        model="gemini-2.0-flash",
        instruction="""
        Propósito e Metas:

        * Use a ferramenta de busca do Google (google_search) para encontrar pelo menos 10 opções de produtos relevantes sobre o assunto fornecido pelo usuário.
        * Priorize produtos que estejam disponíveis para compra em lojas online ativas no último mês.
        * Filtre os resultados, evitando produtos com muitas avaliações negativas ou sem avaliações, buscando alternativas de maior qualidade e satisfação do cliente.
        * Forneça uma lista clara e concisa das opções de produtos encontradas, incluindo informações como nome do produto, preço (se disponível), loja onde pode ser encontrado e um breve resumo de suas principais características.

        Comportamentos e Regras:

        1) Busca Inicial:

        a) Receba o assunto do produto desejado do usuário.
        b) Utilize a ferramenta 'google_search' com termos de busca relevantes para o assunto.
        c) Realize múltiplas buscas e refine os termos conforme necessário para obter uma variedade de opções.

        2) Seleção e Filtragem de Produtos:

        a) Analise os resultados da busca, priorizando lojas online que estejam ativas no último mês.
        b) Verifique a disponibilidade dos produtos para compra.
        c) Avalie a reputação dos produtos com base em avaliações de usuários e comentários.
        d) Descarte produtos com muitas avaliações negativas ou sem avaliações, buscando alternativas.
        e) Selecione pelo menos 10 opções de produtos relevantes que atendam aos critérios de disponibilidade e avaliação.

        3) Apresentação dos Resultados:

        a) Apresente as 10 opções de produtos de forma clara e organizada (por exemplo, em uma lista numerada).
        b) Para cada produto, inclua o nome, preço (se disponível), nome da loja e um breve resumo de suas características.
        c) Se possível, inclua um link direto para a página do produto na loja online.
        d) Mantenha a objetividade e evite expressar opiniões pessoais sobre os produtos.

        Tom Geral:

        * Seja eficiente e preciso na busca por produtos.
        * Mantenha um tom profissional e informativo.
        """,
        description="Agente que busca produtos no Google",
        tools=[google_search]
    )

    entrada_do_agente_explorador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    lancamentos = call_agent(explorador, entrada_do_agente_explorador)
    return lancamentos

In [67]:
################################################
# --- Agente 2: Especialista em consumo sustentável --- #
################################################
def agente_especialista_ambiental(topico, lista_de_produtos):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        instruction="""
        Propósito e Metas:
        * Atuar como um especialista em consumo sustentável, fornecendo avaliações detalhadas de produtos com base em informações pesquisadas no Google (google_search) sobre as empresas produtoras.
        * Investigar e analisar os impactos ambientais, sociais e a ética de cada empresa.
        * Para cada produto na lista fornecida pelo usuário, buscar informações relevantes sobre a empresa no Google para determinar sua sustentabilidade.
        * Incluir uma avaliação ambiental e social concisa para cada produto, com base nas informações encontradas sobre a empresa.
        * Atribuir uma nota de 0 a 10 para cada empresa/produto, refletindo sua sustentabilidade geral.
        * Ao final da análise, classificar as opções de produtos com base em sua sustentabilidade, do mais para o menos sustentável.
        * Recomendar a compra de produtos de empresas que obtiverem uma nota igual ou superior a 7, justificando a recomendação com base nos critérios de sustentabilidade avaliados.

        Comportamentos e Regras:

        1) Análise da Lista de Produtos:
            a) Receber a lista de produtos fornecida pelo usuário.
            b) Para cada produto na lista, identificar a empresa produtora.
            c) Realizar uma pesquisa no Google (ferramenta google_search) utilizando o nome da empresa para coletar informações sobre suas práticas ambientais, sociais e éticas.

        2) Avaliação e Pontuação:
            a) Com base nas informações coletadas, avaliar os impactos ambientais da empresa (por exemplo, emissões de carbono, uso de recursos naturais, gestão de resíduos, poluição).
            b) Avaliar os aspectos sociais da empresa (por exemplo, condições de trabalho, direitos humanos, envolvimento com a comunidade, diversidade e inclusão).
            c) Considerar a ética da empresa (por exemplo, transparência, responsabilidade corporativa, histórico de controvérsias).
            d) Sintetizar as informações em uma avaliação ambiental e social concisa para cada produto.
            e) Atribuir uma nota de 0 a 10 à empresa/produto, onde notas mais altas indicam maior sustentabilidade.

        3) Ranking e Recomendação:
            a) Classificar todos os produtos analisados com base nas notas de sustentabilidade atribuídas, apresentando primeiro os mais sustentáveis.
            b) Recomendar a compra dos produtos de empresas que obtiverem uma nota igual ou superior a 7.
            c) Fornecer uma breve justificativa para cada recomendação, destacando os aspectos positivos de sustentabilidade da empresa.

        Tom Geral:

        * Adotar um tom profissional, informativo e objetivo.
        * Apresentar as avaliações e recomendações de forma clara e concisa.
        * Evitar opiniões pessoais ou preferências, baseando as análises em dados e informações encontradas.
        * Ser direto e eficiente na apresentação dos resultados.
        """,
        description="Agente que busca mais informaçoes sobre práticas sustentáveis de empresas",
        tools=[google_search]
    )

    entrada_do_agente_especialista_ambiental = f"Tópico:{topico}\Lista de produtos: {lista_de_produtos}"
    # Executa o agente
    praticas_sustentaveis = call_agent(planejador, entrada_do_agente_especialista_ambiental)
    return praticas_sustentaveis

In [68]:
######################################
# --- Agente 3: Agente especialista em economia --- #
######################################
def agente_especialista_economia(topico, lista_de_produtos):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
        Propósito e Objetivos:
        * Analisar uma lista de produtos fornecida pelo usuário.
        * Realizar buscas no Google (google_search) para encontrar avaliações de outros usuários sobre esses produtos.
        * Avaliar a qualidade e o preço de cada produto com base nas informações encontradas.
        * Recomendar a compra dos produtos que apresentam o melhor custo-benefício.
        * Apresentar um ranking dos produtos, priorizando aqueles com a melhor relação entre qualidade e preço.

        Comportamentos e Regras:

        1) Análise Inicial:
            a) Receber a lista de produtos do usuário.
            b) Para cada produto na lista, realizar uma busca no Google utilizando a função 'google_search' para encontrar avaliações e informações relevantes.
            c) Analisar as avaliações encontradas, prestando atenção às opiniões sobre a qualidade, durabilidade, funcionalidades e outros aspectos relevantes de cada produto.
            d) Coletar informações sobre o preço atual de cada produto, buscando diferentes fontes se necessário.

        2) Avaliação de Custo-Benefício:
            a) Comparar a qualidade (com base nas avaliações) com o preço de cada produto.
            b) Identificar os produtos que oferecem a melhor combinação de boa qualidade por um preço justo ou acessível.
            c) Considerar diferentes necessidades e prioridades que os usuários podem ter ao avaliar o custo-benefício.

        3) Apresentação dos Resultados:
            a) Apresentar uma lista ranqueada dos produtos, onde os primeiros itens são aqueles com o melhor custo-benefício.
            b) Fornecer uma breve justificativa para o ranking de cada produto, explicando os motivos pelos quais foram considerados com bom ou mau custo-benefício.
            c) Utilizar linguagem clara e objetiva, evitando jargões técnicos desnecessários.
            d) Responder de forma concisa e direta às perguntas do usuário sobre o ranking ou os produtos.

        Tom Geral:

        * Adotar um tom profissional e experiente como planejador financeiro.
        * Ser objetivo e imparcial na análise dos produtos.
        * Priorizar a clareza e a utilidade das informações fornecidas ao usuário.
        * Manter uma postura de especialista em custo-benefício.
            """,
        description="Agente especialista em economia"
    )
    entrada_do_agente_especialista_economia = f"Tópico: {topico}\nLista de produtos: {lista_de_produtos}"
    # Executa o agente
    aspectos_economicos = call_agent(redator, entrada_do_agente_especialista_economia)
    return aspectos_economicos

In [71]:
##########################################
# --- Agente 4: Decisão final --- #
##########################################
def agente_decisor(topico, praticas_sustentaveis, aspectos_economicos, orcamento):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Propósito e Objetivos:
            * Analisar criticamente opções de produtos para o usuário, considerando práticas sustentáveis (ambientais, sociais e éticas), aspectos financeiros e econômicos.
            * Gerar uma lista com o nome do produto, preço, fabricante, parecer ambiental e social, avaliação de preço atrativo e recomendação de compra.
            * Incluir link da homepage da empresa para que o cliente consiga encontrar mais informações sobre o produto

            Comportamentos e Regras:

            1) Análise de Produtos:

            a) Para cada produto apresentado pelo usuário, investigar as práticas de sustentabilidade da empresa fabricante em relação ao meio ambiente, à sociedade e à ética.
            b) Avaliar se o preço do produto é atrativo em relação ao seu valor e às alternativas disponíveis.
            c) Considerar o orçamento disponível do usuário ao fazer a recomendação final.

            2) Apresentação da Análise:

            a) Gerar uma tabela clara e concisa para cada produto analisado, contendo:
                i) Nome do Produto
                ii) Preço
                iii) Nome da Empresa Fabricante
                iv) Parecer Ambiental (e.g., 'boas práticas', 'esforços limitados', 'preocupante')
                v) Parecer Social e Ético (e.g., 'excelente engajamento social', 'cumpre o básico', 'questões éticas identificadas')
                vi) Avaliação de Preço ('atrativo', 'justo', 'acima da média')
                vii) Recomendação de Compra ('comprar', 'considerar alternativas', 'não recomendado')

            Tom Geral:

            * Adotar um tom agradável e descontraído com o usuário.
            * Apresentar as recomendações de forma imparcial e transparente, com base na análise realizada e no orçamento do usuário.
            * Utilizar linguagem clara e acessível.
            * Uso de emojis quando possível, porém sem exagero
            """,
        description="Agente decisor sobre a compra."
    )
    entrada_do_agente_decisor = f"Tópico: {topico}\nInformações sobre práticas sustentáveis do: {praticas_sustentaveis}\nAnalise financeira sobre o produto: {aspectos_economicos}\nOrçamento disponível: {orcamento}"
    # Executa o agente
    lista_de_produtos_revisada = call_agent(revisor, entrada_do_agente_decisor)
    return lista_de_produtos_revisada

In [72]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("Olá, sou o Clau.\nSeu assistente de compras sustentáveis e econômicas.☘️")

# --- Obter o Tópico do Usuário ---
produto = input("O que você gostaria de comprar hoje?")
orcamento = input("Quanto você tem disponível para compra?")


# Inserir lógica do sistema de agentes ################################################
if not produto:
    print("Você esqueceu de digitar o produto")
else:
    print(f"Maravilha! Estou buscando informações sobre o {produto}")

    lista_produtos = agente_explorador(produto, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (Explorador) ---\n")
    display(to_markdown(lista_produtos))
    print("--------------------------------------------------------------")

    praticas_sustentaveis = agente_especialista_ambiental(produto, lista_produtos)
    print("\n--- 📝 Resultado do Agente 2 (Especialista em práticas sustentáveis) ---\n")
    display(to_markdown(praticas_sustentaveis))
    print("--------------------------------------------------------------")

    aspectos_economicos = agente_especialista_economia(produto, lista_produtos)
    print("\n--- 📝 Resultado do Agente 3 (Especialista em Economia) ---\n")
    display(to_markdown(aspectos_economicos))
    print("--------------------------------------------------------------")

    decisao = agente_decisor(produto, praticas_sustentaveis, aspectos_economicos, orcamento)
    print("\n--- 📝 Resultado do Agente 4 (Decisor) ---\n")
    display(to_markdown(decisao))
    print("--------------------------------------------------------------")

Olá, sou o Clau.
Seu assistente de compras sustentáveis e econômicas.☘️
Maravilha! Estou buscando informações sobre o Tênis de Corrida 600

--- 📝 Resultado do Agente 1 (Explorador) ---



> Okay, estou pronto para procurar tênis de corrida 600. Vou usar a ferramenta de busca do Google para encontrar pelo menos 10 opções de produtos relevantes que estejam disponíveis para compra em lojas online ativas e com boas avaliações.
> 
> 
> Com base nas minhas buscas, aqui estão algumas opções de tênis de corrida, focando em modelos que se enquadram no que você procura (Tênis de Corrida 600) e informações relevantes de 2025:
> 
> **Opções de Tênis de Corrida:**
> 
> 1.  **Olympikus Corre 4:**
>     *   **Características:** Funciona bem para treinos e corridas de curta e longa distância.
>     *   **Preço médio:** R$ 500.
> 
> 2.  **Nike Pegasus 41:**
>     *   **Características:** Bom amortecimento para corridas de alto impacto.
>     *   **Preço médio:** R$ 899.
> 
> 3.  **Mizuno Wave Rider 27:**
>     *   **Características:** Tênis de alta durabilidade para corredores.
>     *   **Preço médio:** R$ 430.
> 
> 4.  **Fila Speedzone:**
>     *   **Características:** Projetado para proporcionar alta performance.
>     *   **Preço médio:** R$ 600.
> 
> 5.  **Adidas Supernova Rise:**
>     *   **Características:** Bastante amortecimento, desenvolvido para o conforto do corredor. Espuma Dreamstrike Plus.
>     *   **Preço:** Lançado por R$999, mas pode ser encontrado por menos de R$ 600.
> 
> 6.  **ASICS Dynablast 4:**
>     *   **Características:** Bom para quem busca conforto e responsividade.
>     *   **Preço:** Abaixo de R$ 600.
> 
> 7.  **New Balance 880v13/v14:**
>     *   **Características:** Conforto interno, espuma Fresh Foam X.
>     *   **Preço:** Na faixa dos R$ 600.
> 
> 8.  **Fila Float Maxxi 2:**
>     *   **Características:** Bom amortecimento e durabilidade, ideal para iniciantes e pessoas com sobrepeso.
>     *   **Preço:** R$500 - R$550.
> 
> 9.  **Olympikus Corre Grafeno 2:**
>     *   **Características:** Placa de grafeno, solado com borracha Michelin para durabilidade.
>     *   **Preço:** Um pouco mais caro que o Corre 3, mas com tecnologias superiores.
> 
> 10. **ASICS Gel-Pulse 15:**
>     *   **Características:** Modelo de entrada da ASICS, bom custo-benefício para iniciantes, confortável e com bom amortecimento.
>     *   **Preço:** Menos de R$ 400.
> 
> **Outras opções mencionadas nas buscas:**
> 
> *   **Nike Air Zoom Winflo 7/ Pegasus 38/ Revolution 5:** Modelos da Nike com diferentes características, dependendo do seu tipo de pisada e necessidade de amortecimento.
> *   **Adidas Run Falcon 2.0:** Uma opção mais acessível da Adidas.
> *   **Under Armour HOVR Overlap:**  Tênis de corrida unissex da Under Armour.
> *   **Puma Speed 600 2:** Indicado para corrida.
> 
> **Onde Comprar e Considerações:**
> 
> *   **Lojas online:** Centauro, Netshoes, World Tennis, e Amazon são bons lugares para procurar.
> *   **Tipo de Pisada:** É importante saber seu tipo de pisada (neutra, pronada ou supinada) para escolher o tênis ideal.
> *   **Desgaste:** A vida útil de um tênis de corrida é de aproximadamente 600 km.
> *   **Promoções:** Fique de olho nos grupos de promoções de tênis para encontrar os melhores preços.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Especialista em práticas sustentáveis) ---



> Okay, com a sua lista de tênis de corrida em mãos, vou agora analisar a sustentabilidade de cada um, buscando informações sobre as empresas fabricantes.
> 
> 
> Com base nas informações encontradas, aqui está uma avaliação de sustentabilidade para cada um dos tênis de corrida em sua lista:
> 
> **1. Olympikus Corre 4 / Olympikus Corre Grafeno 2:**
> 
> *   **Avaliação Ambiental:** A Olympikus tem demonstrado um forte compromisso com a sustentabilidade. Desde 2021, todos os seus tênis são produzidos com 100% de energia limpa. A empresa também utiliza materiais reciclados em alguns modelos, como o Corre 1 Eco, que usa EVA verde produzido a partir de cana-de-açúcar e poliéster reciclado de garrafas PET. Além disso, a Olympikus possui iniciativas para economia circular e reúso de água em seus processos produtivos.
> *   **Avaliação Social:** A Olympikus investe em programas sociais que visam promover a inclusão e o acesso ao esporte, principalmente para comunidades carentes. A marca também se preocupa com o desenvolvimento de tecnologias e inovações que elevam a performance dos atletas e amantes do esporte.
> *   **Nota:** 8/10
> 
> **2. Adidas Supernova Rise:**
> 
> *   **Avaliação Ambiental:** A Adidas tem um forte compromisso com a sustentabilidade, com diversas iniciativas em andamento. O programa "Parley for the Oceans" é um exemplo notável, transformando plásticos oceânicos em produtos. A Adidas também planeja utilizar apenas poliéster reciclado em todos os seus produtos até 2025 e tem como objetivo reduzir seu impacto ambiental através do uso de materiais reciclados e práticas de produção mais sustentáveis.
> *   **Avaliação Social:** A Adidas demonstra preocupação com a responsabilidade social, adotando práticas éticas e buscando garantir condições de trabalho justas em sua cadeia de fornecimento. A empresa também investe em projetos sociais e iniciativas comunitárias.
> *   **Nota:** 7.5/10
> 
> **3. New Balance 880v13/v14:**
> 
> *   **Avaliação Ambiental:** A New Balance está comprometida com a fabricação de produtos de forma mais responsável, utilizando materiais e métodos sustentáveis. A empresa tem metas ambiciosas para 2030, incluindo o uso de 100% de poliéster preferencial, com 25% de fibras recicladas, e couro livre de cromo ou de agricultura regenerativa. Além disso, a New Balance busca reduzir o uso de substâncias químicas perigosas e alcançar zero resíduos em suas fábricas de calçados.
> *   **Avaliação Social:** A New Balance possui um programa de Liderança Responsável e estabeleceu ações e alianças para promover os direitos humanos e o bem-estar dos trabalhadores em sua cadeia de fornecimento. A empresa também está comprometida em garantir que os trabalhadores tenham acesso a programas de treinamento e desenvolvimento pessoal.
> *   **Nota:** 7/10
> 
> **4. Mizuno Wave Rider 27:**
> 
> *   **Avaliação Ambiental:** A Mizuno está cada vez mais focada em sustentabilidade, com a coleção Wave Neo utilizando materiais reciclados no cabedal e solado. A empresa tem como meta a neutralidade de carbono até 2050 e está implementando medidas para reduzir as emissões de CO2 em seus processos produtivos.
> *   **Avaliação Social:** A Mizuno busca garantir práticas éticas em sua cadeia de fornecimento e apoia iniciativas sociais em comunidades onde atua. A empresa está comprometida em promover o bem-estar de seus funcionários e em contribuir para o desenvolvimento social.
> *   **Nota:** 6.5/10
> 
> **5. Nike Pegasus 41:**
> 
> *   **Avaliação Ambiental:** A Nike tem a iniciativa "Move to Zero", que visa alcançar emissão zero de carbono e resíduos. A empresa utiliza materiais reciclados em muitos de seus produtos, como poliéster reciclado e nylon regenerado. A Nike também busca reduzir o desperdício em suas operações e implementar soluções circulares.
> *   **Avaliação Social:** A Nike tem um forte compromisso com a responsabilidade social corporativa, abordando questões como sustentabilidade, diversidade e inclusão. A empresa promove condições de trabalho justas e apoia comunidades locais através de programas educacionais e esportivos. No entanto, a Nike enfrentou críticas por não atingir totalmente suas metas ambientais e por alegações de "greenwashing".
> *   **Nota:** 6/10
> 
> **6. ASICS Dynablast 4 / ASICS Gel-Pulse 15:**
> 
> *   **Avaliação Ambiental:** As informações sobre as iniciativas de sustentabilidade da ASICS são limitadas. Seria necessário mais pesquisa para determinar o nível de compromisso da empresa com práticas ambientais responsáveis.
> *   **Avaliação Social:** Assim como na avaliação ambiental, as informações sobre as práticas sociais da ASICS são escassas. É importante buscar mais dados para avaliar o impacto social da empresa.
> *   **Nota:** 5/10 (Nota conservadora devido à falta de informações)
> 
> **7. Fila Speedzone / Fila Float Maxxi 2:**
> 
> *   **Avaliação Ambiental:** A Fila demonstra preocupação com a sustentabilidade ambiental através de ações como o uso de formulações concentradas para reduzir o desperdício e otimizar as embalagens, a produção de produtos sem solventes que danifiquem a camada de ozono e a utilização de matérias-primas de origem vegetal em suas formulações. A empresa busca melhorar continuamente seu Sistema de Gestão para a Qualidade e Ambiente.
> *   **Avaliação Social:** A Fila está empenhada em não utilizar trabalho infantil ou escravo e em não praticar discriminações. A empresa apoia projetos de solidariedade e busca construir relações duradouras com seus públicos.
> *   **Nota:** 5/10
> 
> **Ranking de Sustentabilidade (do mais para o menos sustentável):**
> 
> 1.  Olympikus Corre 4 / Olympikus Corre Grafeno 2 (8/10)
> 2.  Adidas Supernova Rise (7.5/10)
> 3.  New Balance 880v13/v14 (7/10)
> 4.  Mizuno Wave Rider 27 (6.5/10)
> 5.  Nike Pegasus 41 (6/10)
> 6.  ASICS Dynablast 4 / ASICS Gel-Pulse 15 (5/10)
> 7.  Fila Speedzone / Fila Float Maxxi 2 (5/10)
> 
> **Recomendações de Compra:**
> 
> *   **Olympikus Corre 4 / Olympikus Corre Grafeno 2 (8/10):** Recomendado devido ao forte compromisso da Olympikus com a sustentabilidade, incluindo o uso de energia limpa na produção, materiais reciclados e programas sociais.
> *   **Adidas Supernova Rise (7.5/10):** Recomendado pelas iniciativas da Adidas em sustentabilidade, como o programa "Parley for the Oceans" e o compromisso de usar apenas poliéster reciclado até 2025.
> *   **New Balance 880v13/v14 (7/10):** Recomendado pelas metas ambiciosas da New Balance para 2030 em relação ao uso de materiais sustentáveis e práticas de produção responsáveis.
> 
> Lembre-se que esta avaliação é baseada nas informações disponíveis e pode não ser completa. Recomendo pesquisar mais sobre as práticas de sustentabilidade de cada empresa antes de tomar uma decisão de compra.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Especialista em Economia) ---



> Com base na sua lista de tênis de corrida e considerando um orçamento de até R$ 600, aqui está uma análise de custo-benefício para te ajudar a escolher:
> 
> **Ranking de Custo-Benefício (Priorizando o Orçamento de R$600):**
> 
> 1.  **ASICS Gel-Pulse 15:**
>     *   **Por que:** Com preço abaixo de R$ 400, é uma excelente opção para iniciantes. Oferece bom amortecimento e conforto, ideal para quem está começando a correr e não quer gastar muito.
>     *   **Ideal para:** Iniciantes, corredores de curta distância, quem busca conforto.
> 
> 2.  **Mizuno Wave Rider 27:**
>     *   **Por que:** Durabilidade é a palavra-chave aqui. Por cerca de R$ 430, você terá um tênis resistente para muitas corridas.
>     *   **Ideal para:** Corredores que buscam durabilidade e bom desempenho em treinos diários.
> 
> 3.  **Fila Float Maxxi 2:**
>     *   **Por que:** Amortecimento e durabilidade por um preço acessível (R$500 - R$550). Uma ótima escolha para iniciantes e pessoas com sobrepeso.
>     *   **Ideal para:** Iniciantes, corredores com sobrepeso, quem precisa de bom amortecimento.
> 
> 4.  **Olympikus Corre 4:**
>     *   **Por que:** Funciona bem para treinos e corridas de curta e longa distância por cerca de R$ 500.
>     *   **Ideal para:** Corredores que buscam versatilidade e bom desempenho em treinos diários.
> 
> 5.  **ASICS Dynablast 4:**
>     *   **Por que:** Abaixo de R$ 600, oferece conforto e responsividade. Bom para quem quer um tênis um pouco mais rápido para treinos e provas.
>     *   **Ideal para:** Corredores que buscam conforto e responsividade em treinos e provas.
> 
> 6.  **Adidas Supernova Rise:**
>     *   **Por que:** Originalmente lançado por R$999, se encontrado por menos de R$ 600, pode ser uma boa opção devido ao seu amortecimento superior com a espuma Dreamstrike Plus.
>     *   **Ideal para:** Corredores que buscam amortecimento e conforto em longas distâncias.
> 
> 7.  **New Balance 880v13/v14:**
>     *   **Por que:** Na faixa dos R$ 600, oferece bom conforto interno com a espuma Fresh Foam X.
>     *   **Ideal para:** Corredores que buscam conforto e bom ajuste para treinos diários.
> 
> 8.  **Fila Speedzone:**
>     *   **Por que:** Projetado para alta performance, mas com preço na faixa de R$ 600. Pode ser uma boa opção se você busca um tênis mais rápido e responsivo.
>     *   **Ideal para:** Corredores experientes que buscam performance e velocidade.
> 
> **Opções que podem ser consideradas (se o orçamento permitir):**
> 
> *   **Olympikus Corre Grafeno 2:** Se você puder gastar um pouco mais, a placa de grafeno e o solado Michelin podem valer o investimento em durabilidade e desempenho.
> *   **Nike Pegasus 41:** Se você prioriza amortecimento de alto impacto e está disposto a gastar um pouco mais, pode ser uma boa opção.
> 
> **Observações Importantes:**
> 
> *   **Tipo de Pisada:** Antes de comprar, é fundamental saber o seu tipo de pisada para escolher o tênis mais adequado.
> *   **Promoções:** Fique de olho em promoções e cupons para conseguir um preço ainda melhor.
> 
> Espero que esta análise te ajude a escolher o tênis ideal para suas corridas!


--------------------------------------------------------------

--- 📝 Resultado do Agente 4 (Decisor) ---



> Olá! 😊 Analisando suas opções de tênis com foco em sustentabilidade, custo-benefício e seu orçamento de R$600, preparei uma tabela com as melhores recomendações pra você!
> 
> | Produto                | Preço (R$) | Fabricante | Parecer Ambiental 🌿 | Parecer Social e Ético 🤝 | Avaliação de Preço 💰 | Recomendação de Compra ✅ | Link da Empresa                                                        |
> | ---------------------- | ---------- | ---------- | --------------------- | ------------------------- | ----------------------- | ------------------------ | --------------------------------------------------------------------- |
> | ASICS Gel-Pulse 15     | <400       | ASICS      | Informações Limitadas  | Informações Limitadas     | Atrativo                | Comprar                  | [ASICS](https://www.asics.com.br/)                                    |
> | Mizuno Wave Rider 27   | 430        | Mizuno     | Crescendo             | Cumpre o Básico          | Justo                   | Comprar                  | [Mizuno](https://www.mizuno.com.br/)                                  |
> | Olympikus Corre 4      | 500        | Olympikus  | Boas Práticas        | Excelente Engajamento     | Justo                   | Comprar                  | [Olympikus](https://www.olympikus.com.br/)                              |
> | Fila Float Maxxi 2     | 500-550    | Fila       | Cumpre o Básico        | Cumpre o Básico          | Justo                   | Comprar                  | [Fila](https://www.fila.com.br/)                                      |
> | ASICS Dynablast 4     | <600       | ASICS      | Informações Limitadas  | Informações Limitadas     | Justo                   | Comprar                  | [ASICS](https://www.asics.com.br/)                                    |
> | Adidas Supernova Rise  | <600*      | Adidas     | Boas Práticas        | Cumpre o Básico          | Atrativo (se <R$600)   | Considerar Alternativas | [Adidas](https://www.adidas.com.br/)                                   |
> | New Balance 880v13/v14 | 600        | New Balance| Boas Práticas        | Cumpre o Básico          | Justo                   | Considerar Alternativas | [New Balance](https://www.newbalance.com.br/)                           |
> 
> *   **Observações:**
> 
>     *   **ASICS Gel-Pulse 15**: Ideal para iniciantes e quem busca economia.
>     *   **Mizuno Wave Rider 27**: Ótima durabilidade para treinos diários.
>     *   **Olympikus Corre 4**: Versátil para diferentes distâncias.
>     *   **Fila Float Maxxi 2**: Bom amortecimento para iniciantes e pessoas com sobrepeso
>     *   **ASICS Dynablast 4**: Para quem busca conforto e responsividade.
>     *   **Adidas Supernova Rise**: Se encontrar abaixo de R$600, pode valer a pena pelo amortecimento.
>     *   **New Balance 880v13/v14**: Conforto e bom ajuste para treinos diários.
> 
> Espero que ajude na sua escolha! 😉 Se tiver mais dúvidas, é só perguntar! 😊
> 


--------------------------------------------------------------
